In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [ ]:
# pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
# pip install gensim

In [ ]:
# pip install spacy

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving household_power_consumption.txt to household_power_consumption.txt


In [ ]:
df = pd.read_csv('household_power_consumption.txt', sep=';', low_memory=False)

In [ ]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [ ]:
df.columns

Index(['Date', 'Time', 'Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3'],
      dtype='object')

In [ ]:
# Garder uniquement les colonnes 'Date' et 'Global_active_power'
df = df[['Date', 'Global_active_power']]

In [ ]:
# Vérifier les premières lignes après sélection des colonnes
print(df.head())

         Date Global_active_power
0  16/12/2006               4.216
1  16/12/2006               5.360
2  16/12/2006               5.374
3  16/12/2006               5.388
4  16/12/2006               3.666


-->prédire la consommation du lendemain (c'est-à-dire le Global_active_power du jour suivant) à partir des données des jours précédents.

 fonction shift(-1) de pandas qui décale les valeurs de la colonne Global_active_power d'une ligne vers le haut, créant ainsi la cible pour chaque ligne.

In [ ]:
# Créer la colonne cible 'Target' pour la consommation du lendemain
df['Target'] = df['Global_active_power'].shift(-1)

In [ ]:
print(df.head())

         Date Global_active_power Target
0  16/12/2006               4.216  5.360
1  16/12/2006               5.360  5.374
2  16/12/2006               5.374  5.388
3  16/12/2006               5.388  3.666
4  16/12/2006               3.666  3.520


In [ ]:
# Supprimer les lignes avec des valeurs NaN (notament la dernière ligne de la colonne Target)
df = df.dropna()

In [ ]:
# Vérifier s'il y a des valeurs manquantes (NaN)
print(df.isnull().sum())

Date                   0
Global_active_power    0
Target                 0
dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2075258 entries, 0 to 2075257
Data columns (total 3 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Date                 object
 1   Global_active_power  object
 2   Target               object
dtypes: object(3)
memory usage: 63.3+ MB


In [ ]:
# Vérifier s'il y a des caractères non numériques dans les colonnes numériques (errors='coerce' : Cela forcera la conversion des valeurs non numériques en NaN.)
df['Global_active_power'] = pd.to_numeric(df['Global_active_power'], errors='coerce')

In [ ]:
# Vérifier à nouveau les valeurs non numériques
print(df['Global_active_power'].isnull().sum())

25979


In [ ]:
# Remplir les NaN avec la médiane de la colonne
df['Global_active_power'] = df['Global_active_power'].fillna(df['Global_active_power'].median())

# Vérifier les NaN après remplacement
print(df['Global_active_power'].isnull().sum())

0


In [ ]:
# Convertir les colonnes en type numérique
df['Target'] = pd.to_numeric(df['Target'], errors='coerce')

In [ ]:
print(df['Target'].isnull().sum())

25979


In [ ]:
# Convertir les colonnes en type numérique
df['Target'] = pd.to_numeric(df['Target'], errors='coerce')

# Vérifier les valeurs manquantes après conversion
print(df.isnull().sum())

Date                       0
Global_active_power        0
Target                 25979
dtype: int64


In [ ]:
df['Target'] = df['Target'].fillna(df['Target'].median())

In [ ]:
print(df.isnull().sum())

Date                   0
Global_active_power    0
Target                 0
dtype: int64


Normalisation des données

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Appliquer MinMaxScaler pour normaliser les données
scaler = MinMaxScaler(feature_range=(0, 1))

# Sélectionner les colonnes à normaliser
numeric_cols = ['Global_active_power', 'Target']

# Appliquer la normalisation
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Vérifier les premières lignes après normalisation
print(df.head())

         Date  Global_active_power    Target
0  16/12/2006             0.374796  0.478363
1  16/12/2006             0.478363  0.479631
2  16/12/2006             0.479631  0.480898
3  16/12/2006             0.480898  0.325005
4  16/12/2006             0.325005  0.311787


Création des séquences temporelles

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2075258 entries, 0 to 2075257
Data columns (total 3 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Date                 object 
 1   Global_active_power  float64
 2   Target               float64
dtypes: float64(2), object(1)
memory usage: 63.3+ MB


In [ ]:
# Diviser les données en 80% pour l'entraînement et 20% pour le test
train_size = int(0.8 * len(df))  # 80% pour l'entraînement
test_size = len(df) - train_size  # Le reste pour le tes

In [ ]:
# Séparer les caractéristiques (features) et la cible (target)
X = df[['Global_active_power']]  # Par exemple, on garde 'Global_active_power' comme feature
y = df['Target']  # La cible est la colonne 'Target'

In [ ]:
# Diviser X et y en ensemble d'entraînement et de test
X_train, X_test = X[:train_size], y[train_size:]
y_train, y_test = X[:train_size], y[train_size:]

# Vérifier les formes des ensembles de données
print(f"Ensemble d'entraînement : {X_train.shape}, {y_train.shape}")
print(f"Ensemble de test : {X_test.shape}, {y_test.shape}")

Ensemble d'entraînement : (1660206, 1), (1660206, 1)
Ensemble de test : (415052,), (415052,)


In [ ]:
# Diviser l'ensemble d'entraînement en 80% pour l'entraînement et 20% pour la validation
val_size = int(0.2 * len(X_train))  # 20% des données d'entraînement pour la validation

# Diviser X_train et y_train en ensemble d'entraînement et de validation
X_train_final, X_val = X_train[:-val_size], X_train[-val_size:]
y_train_final, y_val = y_train[:-val_size], y_train[-val_size:]

# Vérifier les formes des ensembles de données
print(f"Ensemble d'entraînement final : {X_train_final.shape}, {y_train_final.shape}")
print(f"Ensemble de validation : {X_val.shape}, {y_val.shape}")

Ensemble d'entraînement final : (1328165, 1), (1328165, 1)
Ensemble de validation : (332041, 1), (332041, 1)


In [ ]:
import torch
from torch.utils.data import Dataset

EnergyConsumptionDataset hérite de Dataset et est conçu pour accepter des données X (entrées) et y (cibles) sous forme de tensors PyTorch.
__getitem__ permet de récupérer une paire de entrée et cible pour un index donné.

In [ ]:
class EnergyConsumptionDataset(Dataset):
    def __init__(self, X, y):
        # Convertir les données en tensors PyTorch
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

In [ ]:
 def __getitem__(self, idx):
        # Retourner les données d'entrée (X) et la cible (y) pour l'index donné
        return self.X[idx], self.y[idx]

 DataLoaders :

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
# Créer des datasets PyTorch pour l'entraînement, la validation et le test
train_dataset = EnergyConsumptionDataset(X_train_final, y_train_final)
val_dataset = EnergyConsumptionDataset(X_val, y_val)
test_dataset = EnergyConsumptionDataset(X_test, y_test)

# Créer des DataLoaders pour entraîner le modèle
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # Mélanger les données d'entraînement
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)  # Pas de mélange pour la validation
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  # Pas de mélange pour le test

# Vérifier le nombre d'éléments dans chaque DataLoader
print(f"Nombre de lots dans le DataLoader d'entraînement : {len(train_loader)}")
print(f"Nombre de lots dans le DataLoader de validation : {len(val_loader)}")
print(f"Nombre de lots dans le DataLoader de test : {len(test_loader)}")

ValueError: could not determine the shape of object type 'DataFrame'